## UI for CAUT Deception Detection

In [1]:
#mediapipe processing 1 video
import os
import cv2
import traceback
import numpy as np
import mediapipe as mp
import math

from MediaPipe_Processing_single_video import *

In [2]:
#openFace processing one video
def process_video_openface(vid_path):
    test_video_path = vid_path
    video_prediction = detector.detect_video(test_video_path, skip_frames=24)
    vid_mean = video_prediction.mean()
    vid_mean_df = vid_mean.to_frame()
    vid_mean_df = vid_mean_df.transpose()
    vid_mean_df = vid_mean_df[['AU01','AU02','AU04','AU05','AU06','AU07','AU09','AU10','AU11','AU12','AU14','AU15','AU17','AU20','AU23','AU24','AU25','AU26','AU28','AU43','anger','disgust','fear','happiness','sadness','surprise','neutral']]
    return vid_mean_df

In [3]:
#predictions for the video
import _pickle as cPickle

def DetectDeception(vid_path,mode):
    if mode == "OpenFace":
        new_X = process_video_openface(vid_path)
        with open('C:\\Work\\606Capstone\\Video_chunks\\Models\\OpenFaceAverage_RFR.pickle', 'rb') as f:
            rf = cPickle.load(f)
    else:
        new_X = process_video_mediapipe(vid_path, required_fps=90)
        with open('C:\\Work\\606Capstone\\Video_chunks\\Models\\MediaPipeSequential_RFR.pickle', 'rb') as f:
            rf = cPickle.load(f)

    preds = rf.predict(new_X)
    return preds[0]

In [ ]:
#predictions for the video
from utils import CautDataloaderRegular
import os
import pandas as pd
import math

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
    
def RFR_Model(vid_path,mode):
    if mode == "OpenFace":
        approach_type = "average"
        data_dir = "C:\\Work\\606Capstone\\Video_chunks\\Excel\\"
    else:
        approach_type = "sequential"
        data_dir = "C:\\Work\\606Capstone\\Video_chunks\\MediaPipe\\"
        
    X_y_data = CautDataloaderRegular.get_X_y_TrainTest(csv_path="C:\\Work\\606Capstone\\Video_chunks\\CSV\\",
                                                       data_dir=data_dir,
                                                       data_mode=mode,
                                                       approach_type=approach_type,
                                                       verbose=True)
    
    X_train, y_train = X_y_data[0], X_y_data[1]
    
    if(mode == "OpenFace"):
        y_video = process_video_openface(vid_path)
    else:
        video_mediapipe = process_video_mediapipe(vid_path, required_fps=90)
        y_video = video_mediapipe.reshape((-1, video_mediapipe.shape[-1]))
    
    # Setup model:
    #fitting and evaluating
    print(f"Creating the model")
    rf = RandomForestClassifier(n_estimators=120)
    
    # fit the model:
    print(f"Fitting the model")
    rf.fit(X_train, y_train)
    
    # predict on test data:
    print(f"Shape of ")
    y_pred = rf.predict(y_video)
    print(f"Predictions: {y_pred}")
    return y_pred[0]

In [4]:
#plot the graph for emotions
def Plot_Emotions(vid_path):
    test_video_path = vid_path
    video_prediction = detector.detect_video(test_video_path, skip_frames=24)
    vid_mean = video_prediction.mean()
    vid_mean_df = vid_mean.to_frame()
    vid_mean_df = vid_mean_df.transpose()
    vid_to_plot = vid_mean_df[['anger','disgust','fear','happiness','sadness','surprise','neutral']].transpose().reset_index()
    trace = go.Bar(x=vid_to_plot[vid_to_plot.columns[0]], y=vid_to_plot[vid_to_plot.columns[1]], 
               marker={'color': vid_to_plot[vid_to_plot.columns[1]], 'colorscale': 'Blugrn'})
    layout = go.Layout(title='Emotions in the Video', width=500, height=450)
    fig = go.Figure(data=[trace], layout=layout)
    return fig

In [5]:
#detector for Openface
from tqdm import tqdm
from feat import Detector

detector = Detector()
detector

feat.detector.Detector(face_model=retinaface, landmark_model=mobilefacenet, au_model=xgb, emotion_model=resmasknet, facepose_model=img2pose)

In [7]:
import dash
from dash import dcc, html, Input, Output, State
import dash_daq as daq
import base64
import os
from werkzeug.utils import secure_filename

ss = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=ss, assets_external_path='./assets/')

ASSET_DIR = "assets"

app.layout = html.Div([
    #heading
    html.H1("DECEPTION DETECTION", style={'textAlign': 'center', 'font-size': '30px'}),
    
    #column 1
    html.Div([
      html.Label("Select an option:", style={'font-size': '24px'}),
        dcc.RadioItems(
            id='video-selector',
            options=[
                {'label': 'Select a video from dropdown', 'value': 'dropdown'},
                {'label': 'Upload a video', 'value': 'upload'},
            ]
        ),
        html.Div([
            dcc.Dropdown(id='file-list',style={'width': '250px'}),
        ], id="dropdown-div", style={'display': 'none'}),
        html.Div([
            dcc.Upload(
            id='upload-video',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select a Video')
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        ], id="upload-div", style={'display': 'none'}),  
    ], style={'display': 'inline-block','vertical-align': 'top'}),
    html.Br(),
    html.Div(id='output-graph'),
    
    #column 2
    html.Div([
      html.Div([
            html.Div([
                html.Video(id='video-player', controls=True, 
                           style={
                                "width": "50%",
                               "height": "65%",# set the width of the video frame
                                "position": "absolute",  # set the position to absolute
                                "left": "40%",  # set the left margin to center the video
                                "top": "50px"  # set the top margin to 50 pixels from the top
                                }
                          )
            ])
        ], id='video-div', style={'display': 'none'}),
        html.Div([
            html.Label('MediaPipe', style={
                'display': 'block',
                'text-align': 'center',
                'position': 'absolute',
                'left': '93%',
                'top': '75px',
                'font-size': '18px'
            }),
            daq.BooleanSwitch(
                id='my-boolean-switch', 
                on=False, 
                color='#1c4a60', 
                vertical=True, 
                labelPosition='bottom',
                style={
                    'position': 'absolute',
                    'left': '93%',
                    'top': '110px'
                }
            ),
            html.Label('OpenFace', style={
                'display': 'block',
                'text-align': 'center',
                'position': 'absolute',
                'left': '93%',
                'top': '150px',
                'font-size': '18px'
            })
        ], id='toggle-div', style={'display': 'none'}),
        html.Div([
            html.Button('DETECT', id='play-button', style={
                                "position": "absolute",
                                "left": "60%",
                                "top": "550px"
                                #'background-color': '#AFE1AF'
                                })
        ], id="detect-div", style={'font-size': '24px', 'display': 'none'}),
        html.Div(id='output', style={
                                "position": "absolute",
                                "left": "55%",
                                "top": "600px", 
                                'font-size': '24px'
                                })  
    ],style={'display': 'inline-block','vertical-align': 'top','margin-left': '100px'})
])

# Define the callback to list the files in the asset folder
@app.callback(
    Output('file-list', 'options'),
    [Input('file-list', 'contents')])
def update_file_list(contents):
    # List the files in the asset folder
    file_list = os.listdir(ASSET_DIR)
    options = [{'label': f, 'value': f} for f in file_list]
    return options

#callback to display the video block once source path is updated
@app.callback(
    Output('video-div', 'style'),
    [Input('video-player', 'src')])
def update_video_src(value):
    if value:
        return {"display" : "inline-block"}
    else:
        return {"display" : "none"}
    
#callback to display the toggle switch
@app.callback(
    Output('toggle-div', 'style'),
    [Input('video-player', 'src')])
def update_video_src(value):
    if value:
        return {"display" : "inline-block"}
    else:
        return {"display" : "none"}

#callback to update the path of the video
@app.callback(
    Output('video-player', 'src'),
    [Input('video-selector', 'value'),
     Input('file-list', 'value'),
     Input('upload-video', 'contents')],
    State('upload-video', 'filename')
)
def upload_file(value, filelistvalue, content, filename):
    if value == "dropdown":
        if filelistvalue:
            src = os.path.join(ASSET_DIR, secure_filename(filelistvalue))
            return src
        else:
            return ""
    else:
        if content is not None:
            video_path = os.path.join(ASSET_DIR, secure_filename(filename))
            content_type, content_string = content.split(',')
            decoded_content = base64.b64decode(content_string)
            with open(video_path, 'wb') as f:
                f.write(decoded_content)
            return video_path
        else:
            return ""
        
#callback to either display dropdown or display upload option
@app.callback(
    [Output('dropdown-div', 'style'),Output('upload-div', 'style')],
    [Input('video-selector', 'value')]
)
def update_video_src(value):
    if value == "dropdown":
        return ({"display" : "inline-block"},{"display" : "none"})
    else:
        return ({"display" : "none"},{"display" : "inline-block"})
    
#call back for displaying the graph
@app.callback(
    Output('output-graph', 'children'),
    [Input('video-player', 'src')]
)
def update_graph(input_value):
    fig = Plot_Emotions(input_value)
    return dcc.Graph(figure=fig)


# Define the callback for the detect button
@app.callback(
    Output('output', 'children'),
    [Input('play-button', 'n_clicks'),
    Input('video-player', 'src'),
    Input('my-boolean-switch', 'on')])
def play_video(n_clicks, src, value):
    if n_clicks is None:
        return f""
    else:
        if value:
            detector = "MediaPipe"
        else:
            detector = "OpenFace"
        v = DetectDeception(src, detector)
        if v:
            return f"The person is lying."
        else:
            return f"The person is saying the truth."
    return src

#callback to display the detect button
@app.callback(
    Output('detect-div', 'style'),
    [Input('video-player', 'src')])
def update_video_src(value):
    if value:
        return {'textAlign': 'center',"display" : "block"}
    else:
        return {'textAlign': 'center',"display" : "none"}
    
# Reset n_clicks when a different video is selected
@app.callback(
    Output('play-button', 'n_clicks'),
    [Input('video-player', 'src')]
)
def reset_n_clicks(src):
    return None

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
    #app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: on


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.12s/it]
C:\Users\hinal\AppData\Local\Temp/ipykernel_11516/405049184.py:5: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


----------------------
Video Capture Path: assets\trial_truth_043_004.mp4
